In [2]:
import pandas as pd
import numpy as np
import torch
import collections, re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
!pip3 install nltk
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df=pd.read_csv("/content/drive/My Drive/NLP project/fake_job_postings.csv",encoding='latin-1') #you need to change the path

In [0]:
import re
from nltk.corpus import stopwords
BAD_SYMBOLS_RE = re.compile('[^a-z ]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text
    

In [0]:
df[['company_profile','description']]=df[['company_profile','description']].replace({np.NAN: "Not provided"})
df['company_profile']=df['company_profile'].astype(str)
df['company_profile'] = df['company_profile'].apply(clean_text)
df['description']=df['description'].astype(str)
df['description'] = df['description'].apply(clean_text)

In [21]:
df[['company_profile','description']]

,company_profile,description
0,food weve created groundbreaking awardwinning ...,food fastgrowing james beard awardwinning onli...
1,seconds worlds cloud video production service ...,organised focused vibrant awesomedo passion cu...
2,valor services provides workforce solutions me...,client located houston actively seeking experi...
3,passion improving quality life geography heart...,company esri environmental systems research in...
4,spotsource solutions llc global human capital ...,job title itemization review managerlocation f...
...,...,...
17875,vend looking awesome new talent come join us y...,case first time youve visited website vend awa...
17876,weblinc ecommerce platform services provider f...,payroll accountant focus primarily payroll fun...
17877,provide full time permanent positions many med...,experienced project cost control staff enginee...
17878,provided,nemsia studios looking experienced visualgraph...


In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(df)), df['fraudulent'],
                                                    stratify=df['fraudulent'], 
                                                    test_size=0.3,random_state=0)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(np.array(train_indices).reshape(-1, 1), df.iloc[train_indices]['fraudulent'])
X_resampled=X_resampled.flatten()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
train_indices=X_resampled

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1,1))
X = vectorizer.fit_transform(df["description"]).toarray()

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
X_train=X[train_indices]
X_test=X[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[4847  257]
 [  59  201]]
0.5598885793871866
0.9410887397464579


In [64]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[5087   17]
 [ 109  151]]
0.705607476635514
0.9765100671140939


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1,1))
X = vectorizer.fit_transform(df["description"]).toarray()
X_train=X[train_indices]
X_test=X[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score
classifier = LogisticRegression(random_state=0,max_iter=4000)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[4933  171]
 [  73  187]]
0.6051779935275081
0.9545115585384042


In [67]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[5086   18]
 [ 105  155]]
0.7159353348729792
0.977069351230425


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_fatures=300
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['description'].values)
X1 = tokenizer.texts_to_sequences(df['description'].values)
X1 = pad_sequences(X1)
X_train=X1[train_indices]
X_test=X1[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score

classifier = LogisticRegression(random_state=0, max_iter=8000)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[2915 2189]
 [ 107  153]]
0.11760184473481934
0.5719612229679344


In [70]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[5098    6]
 [ 147  113]]
0.5963060686015831
0.9714765100671141


In [73]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1,1))
X = vectorizer.fit_transform(df["company_profile"]).toarray()
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
X_train=X[train_indices]
X_test=X[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[4278  826]
 [   3  257]]
0.38272524199553243
0.8454511558538405
[[4280  824]
 [   6  254]]
0.37967115097159937
0.8452647278150633


In [74]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1,1))
X = vectorizer.fit_transform(df["company_profile"]).toarray()
X_train=X[train_indices]
X_test=X[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score
classifier = LogisticRegression(random_state=0,max_iter=4000)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

[[4276  828]
 [   2  258]]
0.3833580980683507
0.8452647278150633
[[4280  824]
 [   4  256]]
0.38208955223880603
0.8456375838926175


In [75]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_fatures=30000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['company_profile'].values)
X1 = tokenizer.texts_to_sequences(df['company_profile'].values)
X1 = pad_sequences(X1)
X_train=X1[train_indices]
X_test=X1[test_indices]
Y_train=df['fraudulent'][train_indices]
Y_test=df['fraudulent'][test_indices]

classifier = LogisticRegression(random_state=0, max_iter=8000)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(Y_test,y_pred))
print(f1_score(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[3530 1574]
 [  19  241]]
0.23228915662650598
0.7030201342281879
[[4279  825]
 [   6  254]]
0.3793876026885736
0.8450782997762863
